In [1]:
#setup data
from dataloader import customDataset, create_dataloader, get_sample_weights
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, RandomResizedCrop
from torch.utils.data import random_split, DataLoader, WeightedRandomSampler

transforms = Compose([
    RandomResizedCrop(size=(256, 256), scale=(0.1, 0.25)),
    # Resize((256, 256)), # Resize images
    # ToTensor(), # Convert images to tensors
    # Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize images
])



c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Directory containing your dataset
directory = r"D:\omer\poverty_mapping_data\clipped_data"

# Calculate sample weights
# sample_weights = get_sample_weights(directory, class_counts)

# Create a WeightedRandomSampler
# sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)


In [3]:
dataset = customDataset(directory=directory, transform=transforms)

# train/test split
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# setup model
from models import load_modified_resnet50

num_classes = 3
model = load_modified_resnet50(num_classes)

c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

# Initialize the model
num_classes = 3
model = load_modified_resnet50(num_classes)
model = model.to(device)

# Specify the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs to train for
num_epochs = 35
best_test_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Train
    for inputs, labels in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs} Training'):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass and optimize
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_dataloader)
    train_acc = correct_predictions / total_predictions

    # Test
    model.eval()
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_dataloader, desc=f'Epoch {epoch+1}/{num_epochs} Testing'):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    test_loss = test_loss / len(test_dataloader)
    test_acc = correct_predictions / total_predictions
    
    torch.save(model.state_dict(), 'last_model.pth')
    # Save the model if test accuracy improves
    if test_acc > best_test_accuracy:
        best_test_accuracy = test_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'New best model saved with accuracy: {test_acc:.4f}')

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

print('Training complete')


Epoch 1/35 Training:   0%|          | 0/53 [00:00<?, ?it/s]c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Epoch 1/35 Testing: 100%|██████████| 23/23 [02:02<00:00,  5.32s/it]


New best model saved with accuracy: 0.7298
Epoch 1/35, Train Loss: 0.7412, Train Accuracy: 0.6858, Test Loss: 1.2370, Test Accuracy: 0.7298


Epoch 2/35 Testing: 100%|██████████| 23/23 [00:17<00:00,  1.29it/s]


New best model saved with accuracy: 0.7451
Epoch 2/35, Train Loss: 0.6362, Train Accuracy: 0.7180, Test Loss: 0.5893, Test Accuracy: 0.7451


Epoch 3/35 Testing: 100%|██████████| 23/23 [00:18<00:00,  1.27it/s]


...

Epoch 33/35, Train Loss: 0.4560, Train Accuracy: 0.8100, Test Loss: 0.6336, Test Accuracy: 0.7284


Epoch 34/35 Testing: 100%|██████████| 23/23 [00:17<00:00,  1.32it/s]


Epoch 34/35, Train Loss: 0.4228, Train Accuracy: 0.8142, Test Loss: 0.7712, Test Accuracy: 0.7061


Epoch 35/35 Testing: 100%|██████████| 23/23 [00:18<00:00,  1.27it/s]


Epoch 35/35, Train Loss: 0.4275, Train Accuracy: 0.8238, Test Loss: 1.7204, Test Accuracy: 0.6128
Training complete


In [7]:
from sklearn.metrics import confusion_matrix, classification_report

model.eval()  # Set the model to evaluation mode

true_labels = []
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)

# Create classification report
report = classification_report(true_labels, predicted_labels)
print("Classification Report:")
print(report)


c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Confusion Matrix:
[[  9  93   0]
 [  0 430   0]
 [  0 186   0]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.09      0.16       102
           1       0.61      1.00      0.76       430
           2       0.00      0.00      0.00       186

    accuracy                           0.61       718
   macro avg       0.54      0.36      0.31       718
weighted avg       0.51      0.61      0.48       718



c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\CITY3\.conda\envs\i-jepaVENV\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
